In this notebook, we will:
    
**1.** Analyze the physical plans of basic aggregations

**2.** Demonstrate the actual computation happening under the hood at each executor.

In [1]:
from pyspark.sql import SparkSession, functions as F

spark = SparkSession.builder.config("spark.sql.shuffle.partitions", 16).getOrCreate()

/home/root/miniconda/envs/py36/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


Following cell creates the demo dataset. It is the same function used in the first notebook. You can skip it.

In [2]:
import pandas as pd
import numpy as np

def create_demo_data(n_products, n_stores, start_date="2021-01-01", end_date="2022-01-01"):
    """Creates demo data, writes it as parquet partitioned by date, reads it and returns the dataframe"""
    dates = pd.date_range(start_date, end_date)
    dates = [str(date)[:10] for date in dates]

    day_index = np.arange(len(dates))
    result = []
    for product in range(n_products):
        for store in range(n_stores):
            sales = np.random.poisson(10, size=len(dates))
            partial_df = (
                pd.DataFrame(dates, columns=["date"])
                .assign(product_id=product)
                .assign(store_id=store)
                .assign(day_index=day_index)
                .assign(sales_quantity=sales)
            )
            result.append(partial_df)
    pdf = pd.concat(result)
    result = spark.createDataFrame(pdf)
    result.repartition("date").write.partitionBy("date").parquet("demo-data", mode="overwrite")
    return spark.read.parquet("demo-data")

In [ ]:
df = create_demo_data(n_products=10, n_stores=10, start_date="2021-01-01", end_date="2021-01-31")

In [ ]:
df.rdd.getNumPartitions()

## Basic Sum:

- We all know this. Let's see the physical plan.

In [ ]:
df.select(F.sum("sales_quantity")).explain()

This is a typical map reduce pattern. The plan translates to:

**1.** read parquet: ``+- FileScan parquet``

**2.** select sales_quantity column: ``+- Project [sales_quantity#19L]``

**3.** Compute the sum at each partition (partial_sum):  ``+- HashAggregate(keys=[], functions=[partial_sum(sales_quantity#19L)])``

**4.** Move the partial results into a single partition ``+- Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#80]``

**5.** Sum up all the partial results to get final result.: ``+- HashAggregate(keys=[], functions=[sum(sales_quantity#50L)])``

In this case we have 16 partitions in the dataframe. At the end of **Step 3**, we will have **a single row at each partition** which only contains the sum of sales_quantity at that partition. Then these results are moved into a single partition.

**Takeaway**:

- Sum requires the shuffle of partial results. However, the amount of data is reduced greatly before this shuffle (to 16 rows in this case). Therefore, this is not a shuffle to worry about.

## Groupby aggregations


### Groupby Sum

In [ ]:
product_sum = df.groupBy("product_id").agg(F.sum("sales_quantity").alias("product_sum"))
product_sum.explain()

- Again, full shuffle is not necessary for groupby aggregations. For ``groupby.sum`` we can find the sum for each partition and then shuffle just the results of each partition and combine them.

- Observe that there are two HashAggregate operations in the physical plan. 1st computes the partial sum, 2nd computes the final sum.

**Note:** We read the physical plan from bottom to top.

The plan is basically:

        Read parquet -> Select necessary columns -> Partial sum -> Shuffle partial results -> Final sum

Spark explains this as follows.

        FileScan parquet -> Project -> HashAggregate -> Exchange hashpartitioning -> HashAggregate
 
``HashAggregate``: Aggregates each partition locally. Decreases the number of rows in a partition.

``Exchange hashpartitioning``: Some cool name for shuffle. Moves the data between executors. Expensive.


**Demonstration:**


**Initial state:**

Executor 1

```
+-------------------------------------------------+
|product_id|store_id|  date      | sales_quantity |
|          |        |            |                |
|    1     |    1   |  2000-01-01|       10       |
|    1     |    1   |  2001-01-02|       12       |
|    2     |    1   |  2000-01-01|       9        |
|    2     |    1   |  2000-01-02|       5        |
|    2     |    2   |  2000-01-01|       3        |
+--------------------------------+----------------+
```

Executor 2

```
+-------------------------------------------------+
|product_id|store_id|  date      | sales_quantity |
|          |        |            |                |
|    1     |    1   |  1999-01-02|       1        |
|    2     |    1   |  1999-01-01|       7        |
|    2     |    1   |  1999-01-02|       13       |
|    3     |    1   |  1999-01-02|       100      |
+--------------------------------+----------------+
```

**1. Aggregate each partition locally:** HashAggregate(keys=[product_id#67], functions=[partial_sum(sales_quantity#71L)])

Executor 1

```
+----------------------+
|product_id|partial_sum|
|          |           |
|    1     |    22     |
|    2     |    17     |
+----------------------+
```

Executor 2

```
+----------------------+
|product_id|partial_sum|
|          |           |
|    1     |    1      |
|    2     |    20     |
|    3     |    100    |
+----------------------+
```

**2. Shuffle partial results**: Exchange hashpartitioning(product_id#67, 16), ENSURE_REQUIREMENTS, [id=#364]

Executor 1

```
+----------------------+
|product_id|partial_sum|
|          |           |
|    1     |    22     |
|    1     |    1      |
+----------------------+
```

Executor 2

```
+----------------------+
|product_id|partial_sum|
|          |           |
|    2     |    20     |
|    2     |    17     |
|    3     |    100    |
+----------------------+
```

**3. Compute final sums:**  HashAggregate(keys=[product_id#67], functions=[sum(sales_quantity#71L)])

Executor 1

```
+----------------------+
|product_id|partial_sum|
|          |           |
|    1     |    23     |
+----------------------+
```

Executor 2

```
+----------------------+
|product_id|partial_sum|
|          |           |
|    2     |    37     |
|    3     |    100    |
+----------------------+
```


### Groupby stddev

Again, full shuffle is not necessary. There is a partial_stddev implementation.



In [ ]:
df.groupBy("product_id").agg(F.stddev("sales_quantity")).explain()

### Groupby countDistinct

In [ ]:
df.groupBy("product_id").agg(F.countDistinct("store_id")).explain()


**1.** Get distinct store/products at each executor

**2.** Shuffle to move store/product pairs to the same partition.

**3.** Again, get distinct store/products at each executor.

**4.** Compute partial counts.

**5.** Shuffle the partial results.

**6.** Compute the final results.

Following demonstration will make it crystal clear.

**Demonstration**


- Assume parquet is partitioned by date. So, all rows for a date is in the same partition
- Assume we have two executors.

**Initial state:**

Executor 1

```
+--------------------------------+
|product_id|store_id|  date      |
|          |        |            |
|    1     |    1   |  2000-01-01|
|    1     |    1   |  2001-01-02|
|    2     |    1   |  2000-01-01|
|    2     |    1   |  2000-01-02|
|    2     |    2   |  2000-01-01|
+--------------------------------+
```
Executor 2

```
+--------------------------------+
|product_id|store_id|  date      |
|          |        |            |
|    1     |    1   |  1999-01-01|
|    1     |    1   |  1999-01-02|
|    2     |    1   |  1999-01-01|
|    2     |    1   |  1999-01-02|
|    2     |    2   |  1999-01-01|
|    3     |    3   |  1999-01-01|
+--------------------------------+
```


**1: Get distinct store/product locally:** HashAggregate(keys=[product_id#67, store_id#68], functions=[])

Executor 1

```
+-------------------+
|product_id|store_id|
|          |        |
|    1     |    1   |
|    2     |    1   |
|    2     |    2   |
+-------------------+
```


Executor 2

```
+-------------------+
|product_id|store_id|
|          |        |
|    1     |    1   |
|    2     |    1   |
|    2     |    2   |
|    3     |    3   |
+-------------------+
```

**2. Shuffle**: Exchange hashpartitioning(product_id#67, store_id#68, 1200), ENSURE_REQUIREMENTS, [id=#370]

- Moves all rows for a store/product pair to the same partition
- Shuffling does not change the total number of rows of the dataframe.
- Note there are some duplicates after this step.

Executor 1:

```
+-------------------+
|product_id|store_id|
|          |        |
|    1     |    1   |
|    1     |    1   |
|    2     |    1   |
|    2     |    1   |
+-------------------+
```


Executor 2

```
+-------------------+
|product_id|store_id|
|          |        |
|    2     |    2   |
|    2     |    2   |
|    3     |    3   |
+-------------------+
```

**3. Again, get distinct store/products locally**: HashAggregate(keys=[product_id#67, store_id#68], functions=[])

Executor 1:

```
+-------------------+
|product_id|store_id|
|          |        |
|    1     |    1   |
|    2     |    1   |
+-------------------+
```


Executor 2

```
+-------------------+
|product_id|store_id|
|          |        |
|    2     |    2   |
|    3     |    3   |
+-------------------+
```

**4. Compute partial counts at each executor**: HashAggregate(keys=[product_id#67], functions=[partial_count(distinct store_id#68)])

We have previously shuffled for product/store we can get partial counts and then add them up later.

```
+------------------------+
|product_id|partial_count|
|          |             |
|    1     |      1      |
|    2     |      1      |
+------------------------+
```

Executor 2

```
+------------------------+
|product_id|partial_count|
|          |             |
|    2     |      1      |
|    3     |      1      |
+------------------------+
```

**5. Shuffle for products:** Exchange hashpartitioning(product_id#67, 1200), ENSURE_REQUIREMENTS, [id=#374]

Executor 1:

```
+------------------------+
|product_id|partial_count|
|          |             |
|    2     |      1      |
|    2     |      1      |
+------------------------+
```

Executor 2

```
+------------------------+
|product_id|partial_count|
|          |             |
|    1     |      1      |
|    3     |      1      |
+------------------------+
```

**6. Sum up the partial counts to get final result.**: HashAggregate(keys=[product_id#67], functions=[count(distinct store_id#68)])

Executor 1

```
+----------------------------+
|product_id|distinct_store_id|
|          |                 |
|    2     |      2          |
+----------------------------+
```

Executor 2

```
+----------------------------+
|product_id|distinct_store_id|
|          |                 |
|    1     |      1          |
|    3     |      1          |
+----------------------------+
```

**Takeaways:**

- ``HashAggregate`` -> ``Exchange`` -> ``HashAggregate`` is a common pattern.

- If it is possible, Spark will do local aggregation first to decrease the amount of data that needs to be shuffled.

- Groupby aggregations are your friend since they avoid the full shuffle.